### This notebook is used for building the datasets for training based on ISONE Load Data

In [15]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np

In [16]:
sheetName = 'NEMASSBOST'

#### Checking for NaN in the original data

In [17]:
df_2016 = pd.read_excel('ISONE Load Data/smd_hourly.xls', sheet_name = sheetName, usecols = [0, 1, 3])
df_2015 = pd.read_excel('ISONE Load Data/smd_hourly(1).xls', sheet_name = sheetName, usecols = [0, 1, 3])
df_2014 = pd.read_excel('ISONE Load Data/2014_smd_hourly.xls', sheet_name = sheetName, usecols = [0, 1, 3])
df_2013 = pd.read_excel('ISONE Load Data/2013_smd_hourly.xls', sheet_name = sheetName, usecols = [0, 1, 3])

print('\n2013 Data contains NaN:', np.any(np.isnan(df_2013.iloc[:,1:].values)))
print('\n2014 Data contains NaN:', np.any(np.isnan(df_2014.iloc[:,1:].values)))
print('\n2015 Data contains NaN:', np.any(np.isnan(df_2015.iloc[:,1:].values)))
print('\n2016 Data contains NaN:', np.any(np.isnan(df_2016.iloc[:,1:].values)))


2013 Data contains NaN: False

2014 Data contains NaN: False

2015 Data contains NaN: False

2016 Data contains NaN: False


In [18]:
df_2016.columns = df_2015.columns
df = pd.concat([df_2013, df_2014, df_2015, df_2016]).reset_index(drop = True)
df['Month'] = pd.DatetimeIndex(df['Date']).month
df['Day'] = pd.DatetimeIndex(df['Date']).day
df['Weekday'] = pd.DatetimeIndex(df['Date']).weekday

In [19]:
def add_prev_info(hours_ahead, demand = df['DEMAND']):
    return demand.shift(hours_ahead)

#### Generate features and labels, then write to file

In [20]:
# ONE DAY BEFORE
df['D1'] = add_prev_info(24)
df['D2'] = add_prev_info(25)
df['D3'] = add_prev_info(26)
df['D4'] = add_prev_info(27)
df['D5'] = add_prev_info(28)
# TWO DAYS BEFORE
df['DD1'] = add_prev_info(46)
df['DD2'] = add_prev_info(47)
df['DD3'] = add_prev_info(48)
df['DD4'] = add_prev_info(49)
df['DD5'] = add_prev_info(50)
# ONE WEEK BEFORE
df['W1'] = add_prev_info(166)
df['W2'] = add_prev_info(167)
df['W3'] = add_prev_info(168)
df['W4'] = add_prev_info(169)
df['W5'] = add_prev_info(170)
# ONE MONTH BEFORE
df['M1'] = add_prev_info(718)
df['M2'] = add_prev_info(719)
df['M3'] = add_prev_info(720)
df['M4'] = add_prev_info(721)
df['M5'] = add_prev_info(722)
# ONE YEAR BEFORE
df['Y1'] = add_prev_info(8758)
df['Y2'] = add_prev_info(8759)
df['Y3'] = add_prev_info(8760)
df['Y4'] = add_prev_info(8761)
df['Y5'] = add_prev_info(8762)
# SOME DAYS BEFORE
df['1D'] = add_prev_info(24)
df['2D'] = add_prev_info(48)
df['3D'] = add_prev_info(72)
df['4D'] = add_prev_info(96)
df['5D'] = add_prev_info(120)
df['6D'] = add_prev_info(144)
df['7D'] = add_prev_info(168)

df['Weekend?'] = (df['Weekday']>4).astype(int)
df['Year'] = pd.DatetimeIndex(df['Date']).year
df = df[df['Year']>2013].drop(columns=['Year'])
df = df.drop([8760,8761])

for col in [x for x in df.columns.tolist() if x not in ['Date', 'Weekday', 'Weekend?']]:
    df = df.loc[df[col]!=0]
    
df = df.reset_index(drop = True)
df.to_csv('Data/'+sheetName+'_data.csv', index = False)

#### Split the data into train/dev sets

In [22]:
regionName = sheetName
csvName = 'Data/'+ regionName + '_data.csv'
df = pd.read_csv(csvName)
df['Year'] = pd.DatetimeIndex(df['Date']).year
df_train = df[df['Year'] == 2014].drop(columns=['Year'])
df_dev = df[df['Year'] == 2015].drop(columns=['Year'])
df_dev = df_dev[df_dev['Month'] < 3]
df_dev = df_dev[:840]
df_dev2 = df[8731+840:8731+840+840]
df_test = df[8731+840*2:8731+840*3]
df_train.to_csv('Data/'+regionName+'_train.csv', index = False)
df_dev.to_csv('Data/'+regionName+'_dev.csv', index = False)
df_dev2.to_csv('Data/'+regionName+'_dev2.csv', index = False)
df_test.to_csv('Data/'+regionName+'_test.csv', index = False)